In [1]:
import requests
import json
from time import sleep

In [2]:
# quiet warnings for local testing
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Get Access Token from Visiting Site

#### TODO add POST /token path to return a valid access token 
    based on email and username  

In [3]:
ACCESS_TOKEN ="AgJ876az32OYpDm7DDg9E1QeDVM1k3zY3qw69X7wDJEV47p491U0C3jrzGpv9Ke2WqKBEqoBEy0aQEF3gxeqoIWbapHkWBEFNkP1"

In [4]:
ACCESS = "?access_token=" + ACCESS_TOKEN

In [5]:
# example options, set custom landing page, delete in 6 minutes
options = {
        "_target": 'http://example.com',
        "_status": 'reserved',
        "ttl": '360'
    } 

# Data Catalog Test

Landing Page : "https://localhost:8080/dc/landing/ark:/99999/fk4RatDC"

In [7]:
dc = {
    "@context": "http://schema.org/",
    "@id": "ark:/99999/fk4RatDC",
    "@type": "DataCatalog",
    "identifier": "ark:/99999/fk4RatDC",
    "name": "Rat Genome Database",
    "url": "http://rgd.mcw.edu/"
}

# with options 
dc_options = {
    'metadata': dc,
    'options': options
}

### Example Put Request

In [8]:
dc_response = requests.put(
    url = "https://localhost:8080/dc/put"+ ACCESS,
    data = json.dumps(dc),
    verify = False
)
dc_response.content

SSLError: HTTPSConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /dc/put?access_token=AgJ876az32OYpDm7DDg9E1QeDVM1k3zY3qw69X7wDJEV47p491U0C3jrzGpv9Ke2WqKBEqoBEy0aQEF3gxeqoIWbapHkWBEFNkP1 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

## Example Put with options

Will create an identifier, but also will delete it in 6 minutes

In [641]:
# put with options
options_response = requests.put(
    url = "https://localhost:8080/dc/put"+ ACCESS,
    data = json.dumps(dc_options),
    verify = False
)
options_response.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"api": {"status": 400, "messsage": "error: bad request - identifier already exists"}, "cache": {"created": {"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}}}'

Test if its still there

In [648]:
dc_get_response = requests.get(
    url = "https://localhost:8080/dc/get/ark:/99999/fk4RatDC" + ACCESS,
    verify = False
)
dc_get_response.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"api": "error: bad request - no such identifier"}'

Wait 6 minutes and test again

In [646]:
sleep(360)
dc_get_response = requests.get(
    url = "https://localhost:8080/dc/get/ark:/99999/fk4RatDC" + ACCESS,
    verify = False
)
dc_get_response.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"api": "error: bad request - no such identifier"}'

### Retrieve the identifier metadata from the cache

Properties about the Data Catalog itself, not members

In [625]:
# get from cache
cache_get_response = requests.get(
    url = "https://localhost:8080/cache/get/ark:/99999/fk4RatDC"+ ACCESS,
    verify = False
)
cache_get_response.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}'

### Delete the DC from the cache

In [628]:
# delete from cache
cache_delete = requests.delete(
    url = "https://localhost:8080/cache/delete/ark:/99999/fk4RatDC"+ACCESS,
    verify = False
)
cache_delete.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}'

In [631]:
# import from ezid to neo4j
cache_import = requests.get(
    url = "https://localhost:8080/dc/import/ark:/99999/fk4RatDC"+ ACCESS,
    verify = False
)
cache_import.content.decode('utf-8')

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


'{"cache": {"imported": {"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}}}'

### Delete from both cache and external API

In [597]:
dc_delete = requests.delete(
    url = "https://localhost:8080/dc/delete/ark:/99999/fk4RatDC"+ACCESS,
    verify = False
)
dc_delete.content

/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'{"cache": {"metadata": {"name": "Rat Genome Database", "guid": "ark:/99999/fk4RatDC", "type": "DataCatalog", "url": "http://rgd.mcw.edu/"}, "message": "Removed from cache"}, "api": {"status_code": 200, "message": "success: ark:/99999/fk4RatDC"}}'

## ARK test

Landing Page https://localhost:8080/ark/landing/ark:/99999/fk4TestDepArk

In [657]:
ark = {
  "@context": "http://schema.org",
  "@id": "ark:/99999/fk4TestDepArk",
  "@type": "CreativeWork",
    "includedInDataCatalog": "ark:/99999/fk4RatDC",
  "identifier": [
   "ark:/88120/r8059v",
   {
      "@type": "PropertyValue",
      "name": "sha-256",
      "value": "cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f"
    }
  ],
  "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v",
  "contentUrl": {"aws": "http://bd2k.ini.usc.edu/assets/all-hands-meeting/minid_v0.1_Nov_2015.pdf"},
  "name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1",
  "author": {
    "@id": "http://orcid.org/0000-0003-2129-5269",
    "@type": "Person",
    "name": "Ian Foster"
  },
  "dateCreated": "2015-11-10T04:44:44.387671Z"
}

ark_options = {
    'metadata': ark,
    'options': options
}


## Put Ark 

In [666]:
local_ark = requests.put(
    url = "https://localhost:8080/ark/put"+ACCESS,
    data = json.dumps(ark),
    verify = False
)
local_ark.content

b'{"api": {"status": 201, "messsage": "success: ark:/99999/fk4TestDepArk"}, "cache": {"created": {"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}}}'

## Put with Options

In [666]:
local_ark = requests.put(
    url = "https://localhost:8080/ark/put"+ACCESS,
    data = json.dumps(ark_options),
    verify = False
)
local_ark.content

b'{"api": {"status": 201, "messsage": "success: ark:/99999/fk4TestDepArk"}, "cache": {"created": {"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}}}'

In [676]:
# get from api
api_response = requests.get(
    url = "https://localhost:8080/ark/get/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(api_response.content)

{'@context': 'http://schema.org',
 '@id': 'ark:/99999/fk4TestDepArk',
 '@type': 'CreativeWork',
 'author': {'id': 'http://orcid.org/0000-0003-2129-5269',
  'name': 'Ian Foster',
  'type': 'Person'},
 'dateCreated': '2015-11-10T04:44:44.387671Z',
 'identifier': ['ark:/88120/r8059v',
  {'name': 'sha-256',
   'type': 'PropertyValue',
   'value': 'cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f'}],
 'includedInDataCatalog': 'ark:/99999/fk4RatDC',
 'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'success': 'ark:/99999/fk4TestDepArk',
 'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}

In [677]:
# get from cache
cache_get_response = requests.get(
    url = "https://localhost:8080/cache/get/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(cache_get_response.content)

{'author': 'Ian Foster',
 'dateCreated': '2015-11-10T04:44:44.387671Z',
 'guid': 'ark:/99999/fk4TestDepArk',
 'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'type': 'Dataset',
 'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}

In [672]:
# delete from cache
cache_delete = requests.delete(
    url = "https://localhost:8080/cache/delete/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(cache_delete.content)

b'{"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}'

In [675]:
# import from ezid to neo4j
cache_import = requests.get(
    url = "https://localhost:8080/ark/import/ark:/99999/fk4TestDepArk"+ACCESS,
    verify = False
)
json.loads(cache_import.content)

{'cache': {'imported': {'author': 'Ian Foster',
   'dateCreated': '2015-11-10T04:44:44.387671Z',
   'guid': 'ark:/99999/fk4TestDepArk',
   'name': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
   'type': 'Dataset',
   'url': 'http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v'}}}

In [406]:
# delete from all
ark_delete = requests.delete(
    url = "http://localhost:8080/ark/delete/ark:/99999/fk4TestDepArk",
    auth = basicAuth
)
ark_delete.content

b'{"cache": {"metadata": {"name": "minid: A BD2K Minimal Viable Identifier Pilot v0.1", "guid": "ark:/99999/fk4TestDepArk", "dateCreated": "2015-11-10T04:44:44.387671Z", "type": "Dataset", "author": "Ian Foster", "url": "http://minid.bd2k.org/minid/landingpage/ark:/88120/r8059v"}, "message": "Removed from cache"}, "api": {"status_code": 200, "message": "b\'success: ark:/99999/fk4TestDepArk\'"}}'

## DOI Test

In [535]:
test_doi = {
  "@context": "http://schema.org",
  "@type": "Dataset",
  "@id": "doi:10.5072/test9999test",
  "identifier": "doi:10.5072/test9999test",
    "includedInDataCatalog": "ark:/99999/fk4RatDC",
  "additionalType": "Data dictionary",
  "name": "A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt",
  "author": {
    "@type": "Organization",
    "name": "The GTEx Consortium"
  },
  "keywords": "gtex, annotation, phenotype, gene regulation, transcriptomics",
  "datePublished": "2017",
  "version": "v7",
  "url": "https://www.gtexportal.org/home/datasets",
  "contentSize": "5.4 Mb",
  "fileFormat": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
  "funder": {
   "@type": "Organization",
   "@id": "https://doi.org/10.13039/100000050",
   "name": "National Heart, Lung, and Blood Institute"
  }
}

In [536]:
dataciteAuth = requests.auth.HTTPBasicAuth("DATACITE.DCPPC", "Deviation6Trickster9landmark")

In [537]:
put_response = requests.put(
    url = "http://localhost:8080/doi/put",
    data = json.dumps(test_doi),
    auth = dataciteAuth
) 

In [538]:
put_response.content

b'{"api": {"status": 401, "messsage": "b\'error: unauthorized\'"}, "cache": {"created": {"name": "A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt", "guid": "doi:10.5072/test9999test", "type": "Dataset", "author": "The GTEx Consortium", "url": "https://www.gtexportal.org/home/datasets"}}}'

In [454]:
get_cache = requests.get(
    url = "http://localhost:8080/cache/get/doi:/0.5072/0000-03VC"
)

In [455]:
get_cache.content

b'null'

In [ ]:
# encode into xml?

In [458]:
# why is postNeo for Doi failing 
from neo4j.v1 import GraphDatabase
driver = GraphDatabase.driver("bolt://localhost:7687", auth=None)

In [519]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        node = tx.run(
            "MATCH (parent:dataCatalog) WHERE parent.guid = $parent "
            "CREATE (node:Doi {guid: $guid, name: $name}) "
            "CREATE (node)-[parentRel:includedIn]->(parent) "
            "RETURN properties(node) ",
            parent = test_doi.get('includedInDataCatalog'),
            guid = test_doi.get('@id'),
            name = test_doi.get('name')
        )

In [520]:
node

In [521]:
#node_data = node.data()
node_data = node.single()

In [522]:
node_data.value()

{'guid': 'doi:0.5072/0000-03VC',
 'name': 'A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt'}

In [525]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        query = tx.run(
            "MATCH (node:Doi) WHERE node.guid = $guid "
            "RETURN properties(node) ",
            guid = test_doi.get('@id')
        )

In [526]:
query.single().value()

{'guid': 'doi:0.5072/0000-03VC',
 'name': 'A data dictionary that describes each variable in the GTEx_v7_Annotations_SubjectPhenotypesDS.txt'}

In [517]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        parent = tx.run(
            "MATCH (parent:dataCatalog) WHERE parent.guid = $parent "
            "RETURN properties(parent) ",
            parent = test_doi.get('includedInDataCatalog'),
        )

In [518]:
parent.single().value()

{'guid': 'ark:/99999/fk4RatDC',
 'name': 'Rat Genome Database',
 'type': 'DataCatalog',
 'url': 'http://rgd.mcw.edu/'}

In [511]:
test_doi.get('includedInDataCatalog')

'ark:/99999/fk4RatDC'

In [527]:
with driver.session() as session:
    with session.begin_transaction() as tx:
        clear = tx.run(
            "MATCH (node:Doi) WHERE node.guid = $guid "
            "DETACH DELETE node ",
            guid = test_doi.get('@id')
        )

In [528]:
clear.data()

[]